#### Intro
We will look at how we can connect to Snowflake from Databricks using Snowflake connector in Databricks.

Snowflake is widely used data warehouse platform and Databricks is a Data Lakehouse Platform. Since we can directly perform Big Data Analytics on Databricks there can be a time, we might need to know how to connect from Databricks to Snowflake.

Also we will read data from a dummy table and see how to write data to snowflake table.

I'm using **Databricks community edition** so I cannot use secrets, instead I have loaded all secrets in a different and run that file from here

Skeleton of Secrets file:

options = {

    "sfUrl":"url",

    "sfUser":"username",

    "sfPassword":"password",

    "sfDatabase":"database name",

    "sfSchema":"schema name",

    "sfWarehouse":"warehouse name",

    }

once you load all these options in a Secrets notebook separately in the same folder level, you can proceed to run following code

**You can utilize free Snowflake trial account to generate all these details**

In [0]:
%run ./Secrets

#### Read

In [0]:
snowflake_table = (spark.read
    .format("snowflake")
    .options(**options) #snowflake connection properties are stored in options as described above
    .option('dbtable',"salemast")
    .load()
)

In [0]:
display(snowflake_table)

SALE_ID,EMPLOYEE_ID,SALE_DATE,SALE_AMT
1,1000,2012-03-08,4500
2,1001,2012-03-09,5500
3,1003,2012-04-10,3500
3,1003,2012-04-10,2500


In [0]:
snowflake_table.printSchema()

root
 |-- SALE_ID: decimal(38,0) (nullable = true)
 |-- EMPLOYEE_ID: decimal(38,0) (nullable = true)
 |-- SALE_DATE: date (nullable = true)
 |-- SALE_AMT: decimal(38,0) (nullable = true)



#### Transform
I have shown here just a little transformation but you can extend these as you need

In [0]:
snowflake_table = snowflake_table.withColumn("SALE_ID",snowflake_table["SALE_ID"].cast('int'))
snowflake_table.printSchema()

root
 |-- SALE_ID: integer (nullable = true)
 |-- EMPLOYEE_ID: decimal(38,0) (nullable = true)
 |-- SALE_DATE: date (nullable = true)
 |-- SALE_AMT: decimal(38,0) (nullable = true)



#### Write
I'm writing a dummy data to snowflake but you can always

In [0]:
spark.range(5).write \
    .format("snowflake") \
    .options(**options) \
    .option("dbtable","dummy") \
    .save()

In [0]:
dummy = spark.read \
    .format("snowflake") \
    .options(**options) \
    .option("dbtable","dummy") \
    .load()

In [0]:
display(dummy)

ID
0
4
3
1
2


#### Write and read to Delta table

In [0]:
dummy.write.format("delta").saveAsTable("sf_ingest_table")

In [0]:
df = spark.read.table("sf_ingest_table")

In [0]:
display(df)

ID
0
4
3
1
2
